In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,335 kB]
Hit:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,428 kB]
Hit:14 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRe

In [2]:
# For connection to Postgres 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-22 17:43:36--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.49MB/s    in 0.2s    

2021-09-22 17:43:36 (4.49 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
user_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", timestampFormat="mm/dd/yy", inferSchema=True)

# Show DataFrame
user_data_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
user_data_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [6]:
user_data_df.printSchema

<bound method DataFrame.printSchema of DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: string, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]>

In [7]:
dropna_df = user_data_df.na.drop(how="any")
dropna_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
dropna_df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [9]:
dropna_df.describe

<bound method DataFrame.describe of DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: string, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]>

In [10]:
# extracting number of rows from the Dataframe
row = dropna_df.count()
   
# extracting number of columns from the Dataframe
col = len(dropna_df.columns)
 
# printing
print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

Dimension of the Dataframe is: (9001052, 15)
Number of Rows are: 9001052
Number of Columns are: 15


In [11]:
# Change the datatype of columns
# We can also make use of datatypes from 
# pyspark.sql.types
from pyspark.sql.types import StringType, DateType, IntegerType
  
new_df = dropna_df \
  .withColumn("star_rating" ,
              dropna_df["star_rating"]
              .cast(IntegerType())) \
  .withColumn("review_date"  ,
              dropna_df["review_date"]
              .cast(DateType())) \
  
new_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
new_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [13]:
new_df.select('review_id')

DataFrame[review_id: string]

In [14]:
type(new_df.select('review_id'))

pyspark.sql.dataframe.DataFrame

In [15]:
new_df.select('review_id').show()

+--------------+
|     review_id|
+--------------+
|R3W4P9UBGNGH1U|
|R15V54KBMTQWAY|
| RY8I449HNXSVF|
|R18TLJYCKJFLSR|
|R1NK26SWS53B8Q|
|R11LOHEDYJALTN|
|R3ALQVQB2P9LA7|
|R3MWLXLNO21PDQ|
|R2L15IS24CX0LI|
|R1DJ8976WPWVZU|
|R3MRWNNR8CBTB7|
|R1DS6DKTUXAQK3|
| RWJM5E0TWUJD2|
|R1XTJKDYNCRGAC|
|R2UZL3DPWEU1XW|
|R2EZXET9KBFFU3|
|R26VY1L1FD3LPU|
|R2SSA4NSFCV18T|
|R1G6333JHJNEUQ|
|R2DRG0UZXJQ0PE|
+--------------+
only showing top 20 rows



In [16]:
# Remove leading and trailing space of the column in pyspark

from pyspark.sql.functions import *

new_df = new_df.withColumn('customer_id', trim(new_df.customer_id))

new_df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [17]:
# extracting number of rows from the Dataframe
row = new_df.count()
   
# extracting number of columns from the Dataframe
col = len(new_df.columns)
 
# printing
print(f'Dimension of the new_df is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

Dimension of the new_df is: (9001052, 15)
Number of Rows are: 9001052
Number of Columns are: 15


In [18]:
review_df = new_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488| 2015-08-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554| 2015-08-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702| 2015-08-31|
|R2L15IS24CX0LI|   12738196|B00HVORET8|     389677711| 2015-08-31|
|R1DJ8976WPWVZU|   15867807|B00HX3G6J6|     299654876| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [21]:
# Create customer_df and rename the columns in customer_df for customer count
from pyspark.sql.functions import col

customer_df = customer_df.select(col("customer_id").alias("customer_id"), col("count(customer_id)").alias("customer_count"))
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1355626|             1|
|   15043009|             1|
|   22770141|             3|
|   45481568|             3|
|   25404882|             2|
|    4697081|             2|
|   13888158|             3|
|   18288553|             2|
|   22279960|             1|
|   25278739|             2|
|   15407188|             1|
|    5030775|            21|
|   13267537|             1|
|   29937804|             1|
|   20919693|             1|
|    8854514|             3|
|   49258595|             8|
|     234036|             1|
|   33346823|             1|
|    7403004|             1|
+-----------+--------------+
only showing top 20 rows



In [31]:
customer_df.count()

5197356

In [32]:
customer_df.distinct().count()

5197356

In [22]:
vine_df = new_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|
|R15V54KBMTQWAY|          4|            0|          0|   N|
| RY8I449HNXSVF|          5|            0|          0|   N|
|R18TLJYCKJFLSR|          5|            0|          0|   N|
|R1NK26SWS53B8Q|          5|            0|          0|   N|
|R11LOHEDYJALTN|          5|            0|          0|   N|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|
|R2L15IS24CX0LI|          5|            0|          0|   N|
|R1DJ8976WPWVZU|          3|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [24]:
from pyspark.sql.functions import isnan
new_df.filter((new_df["review_id"] == "") | new_df["review_id"].isNull() | isnan(new_df["review_id"])).count()

0

In [25]:
from pyspark.sql.functions import isnan
new_df.filter((new_df["customer_id"] == "") | new_df["customer_id"].isNull() | isnan(new_df["customer_id"])).count()

0

In [26]:
product_df = new_df.select(["product_id", "product_title"])
product_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00YL0EKWE|LG G4 Case Hard T...|
|B00XK95RPQ|Selfie Stick Fibl...|
|B00SXRXUKO|Tribe AB40 Water ...|
|B009V5X1CE|RAVPower® Element...|
|B00D93OVF0|Fosmon Micro USB ...|
|B00XVGJMDQ|iPhone 6 Case, Vo...|
|B00KQW1X1C|Nokia Lumia 630 R...|
|B00IP1MQNK|Lumsing 10400mah ...|
|B00HVORET8|iPhone 5S Battery...|
|B00HX3G6J6|HTC One M8 Screen...|
+----------+--------------------+
only showing top 10 rows



In [28]:
product_df.distinct().count()

906482

In [29]:
product_df.count()

9001052

In [33]:
product_df = product_df.distinct()
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B010LVPJH6|LG G Watch Urbane...|
|B00K5ZNXZ4|Minisuit Sporty A...|
|B00S9RBQOK|iPhone 6 Plus Cas...|
|B0116N7GYC|Selfie Stick, Por...|
|B009UNH0CY|Plantronics Voyag...|
|B00L8GFYAG|Eallc New Quality...|
|B00R3LMTI0|Kaleidio [Wallop ...|
|B011R0VG36|Galaxy Note 4 Cas...|
|B00BXX0QVQ|iKross Black Dual...|
|B00F4AYI2M|Incipio DualPro C...|
|B00GPI3OHC|Retevis H-777 2 W...|
|B00Y9ZUVU6|Tiwkich 2 in 1 Du...|
|B00W65SYHS|LG G4 case, Caseo...|
|B00V5OU6CW|S5 Leather case,P...|
|B00LP3FSH6|Escort Coiled Sma...|
|B00MIO2KRC|Black Box G1W-C C...|
|B00V5FZM0M|   KoKo Cases 5/5S !|
|B00T1KO2TA|iPhone 6 & 6S Cas...|
|B00YU9XOTQ|Galaxy S5 Screen ...|
|B00PI7IGHE|Soyan Latest DZ09...|
+----------+--------------------+
only showing top 20 rows

